In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
file_url ='https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
data= pd.read_csv(file_url)

In [4]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [6]:
round(data.describe(),2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


In [9]:
data[['engine', 'engine_units']]=data['engine'].str.split(expand=True)

In [10]:
data['engine']=data['engine'].astype('float32')

In [14]:
data['engine_units'].unique()

array(['CC', nan], dtype=object)

In [16]:
data.drop('engine_units', axis=1, inplace= True)

In [17]:
data[['max_power', 'max_power_units']]=data['max_power'].str.split(expand=True)

In [20]:
def ifFloat(value):
    try:
        num = float(value)
        return num
    except ValueError:
        return np.NaN

data['max_power']=data['max_power'].apply(ifFloat)

In [21]:
data.drop('max_power_units', axis=1, inplace= True)

In [22]:
data[['mileage','mileage_unit']] = data['mileage'].str.split(expand=True) 

In [24]:
data['mileage'] = data['mileage'].astype('float32')

In [28]:
def mile(x):
    if x['fuel'] == 'Petrol':
        return x['mileage']/80.43
    elif x['fuel'] == 'Diesel':
        return x['mileage']/73.56
    elif x['fuel'] == 'LPG':
        return x['mileage']/40.86
    else:
        return x['mileage']/44.23
    
data['mileage'] = data.apply(mile, axis=1)

In [29]:
data.drop('mileage_unit', axis=1, inplace= True)

In [30]:
data['torque'] = data['torque'].str.upper( )

In [32]:
def torque_unit(x):
    if 'NM' in str(x):
        return 'Nm'
    elif 'KGM' in str(x):
        return 'kgm'
data['torque_unit'] = data['torque'].apply(torque_unit) 


In [33]:
data['torque_unit'].fillna('Nm', inplace= True)

In [34]:
def split_num(x):
    x = str(x)
    for i, j in enumerate(x):
        if j not in '0123456789.':
            cut =i
            break
    return x[:cut]
data['torque']= data['torque'].apply(split_num)

In [35]:
data['torque'] = data['torque'].replace('', np.NaN)
data['torque'] = data['torque'].astype('float64') #

In [36]:
def torque_trans(x):
    if x['torque_unit'] == 'kgm':
        return x['torque']*9.8066
    else:
        return x['torque']
data['torque']=data.apply(torque_trans, axis=1)

In [37]:
data.drop('torque_unit', axis=1, inplace=True) 

In [38]:
data['name'] = data['name'].str.split(expand=True)[0] 

In [39]:
data.isna().mean()

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine           0.027190
max_power        0.026575
torque           0.027313
seats            0.027190
dtype: float64

In [40]:
data.dropna(inplace = True)
len(data)

7906

In [41]:
data = pd.get_dummies(data, columns = ['name','fuel','seller_type',
 'transmission','owner'], drop_first=True) 

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('selling_price', axis=1), data['selling_price'], test_size=0.2)

In [55]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
train_pred= model.predict(X_train)
test_pred = model.predict(X_test)

In [57]:
from sklearn.metrics import mean_squared_error
print("train_rmse:", mean_squared_error(y_train, train_pred)**0.5, 
      "test_rmse:", mean_squared_error(y_test, test_pred)**0.5)

train_rmse: 55536.99959243001 test_rmse: 126459.05606941016


In [58]:
from sklearn.model_selection import KFold
data.reset_index(drop=True, inplace = True)

In [60]:
kF = KFold(n_splits=5)

In [61]:
X=data.drop('selling_price', axis =1)
y=data['selling_price']

In [66]:
train_rmse_total =[]
test_rmse_total=[]

for train_index, test_index in kF.split(X):
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    model = RandomForestRegressor(n_estimators = 300, max_depth = 50, min_samples_split = 5, min_samples_leaf = 1, n_jobs=-1)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_rmse = mean_squared_error(y_train, train_pred)**0.5
    test_rmse = mean_squared_error(y_test, test_pred)**0.5
    
    train_rmse_total.append(train_rmse)
    test_rmse_total.append(test_rmse)

In [67]:
print("train_rmse:", sum(train_rmse_total)/5, "test_rmse:",sum(test_rmse_total)/5)

train_rmse: 65840.98010560099 test_rmse: 143403.4660635936
